In [1]:
# Creating some synthetic data for fine-tuning 
# We are going to create serveral characters for video games in here: 

import os
import openai
import pandas as pd

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Define input values
ages = ['30', '40', '50']
genders = ['man', 'woman']
superpowers = ['read in the thoughts', 'turning lead into gold', 'immortality'] 

# Create an empty list to store results
results = []

# Loop through input values
record_number = 1
for age in ages:
    for gender in genders:
        for power in superpowers:
            # Define the prompt with placeholders
            prompt = f"Imagine a complete and detailed description of a {age}-year-old {gender} fictional character who has the superpower of {power}. Write out the entire description in a maximum of 100 words in great detail."

            # Generate the response using OpenAI's GPT-3 model
            response = openai.Completion.create(
                model="text-davinci-003",
                prompt=prompt,
                temperature=0.7,  # Adjust temperature for response creativity
                max_tokens=100,  # Limit response to 100 tokens (adjust as needed)
                n=1,  # Generate a single response
                stop=None,  # Let the model determine when to stop
            )

            # Extract and store the response text
            response_text = response.choices[0].text.strip()

            # Create a subprompt string
            subprompt = f"{age}, {gender}, {power}"

            # Store the results in a dictionary
            result = {
                'record number': record_number,
                'age': age,
                'gender': gender,
                'power': power,
                'prompt': prompt,
                'subprompt': subprompt,
                'response': response_text
            }

            results.append(result)

            # Increment the record number
            record_number += 1

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
df.to_csv("character_descriptions.csv", index=False)


In [2]:
# Now we use the former data to finetune a model 

import pandas as pd
import openai
import subprocess

# read the previous file with the generated data 
df = pd.read_csv("character_descriptions.csv")

# now we are using the parameters of the prompt (subprompt) as final prompt for the training. 
prepared_data = df.loc[:,['subprompt','response']]
prepared_data.rename(columns={'subprompt':'prompt', 'response':'completion'}, inplace=True)
prepared_data.to_csv('prepared_data.csv',index=False)

## prepared_data.csv --> prepared_data_prepared.json
subprocess.run('openai tools fine_tunes.prepare_data --file prepared_data.csv --quiet'.split())

## Start fine-tuning
subprocess.run('openai api fine_tunes.create --training_file prepared_data_prepared.jsonl --model davinci --suffix "SuperHero"'.split())

CompletedProcess(args=['openai', 'api', 'fine_tunes.create', '--training_file', 'prepared_data_prepared.jsonl', '--model', 'davinci', '--suffix', '"SuperHero"'], returncode=0)

In [ ]:
import openai
import time

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Function to retrieve the latest fine-tune job for a model
def get_latest_fine_tune_job(model_name):
    response = openai.FineTune.list(model=model_name, per_page=1)
    if response.data:
        return response.data[0]
    else:
        return None

# Function to check the status of a fine-tuning job
def check_fine_tune_status(fine_tune_id):
    response = openai.FineTune.retrieve(fine_tune_id)

    # Print relevant information about the fine-tuning job
    print("Fine-tune ID:", response.id)
    print("Status:", response.status)
    print("Progress:", response.progress)

    if response.status == "succeeded":
        print("Fine-tuning completed successfully!")
        print("Model ID:", response.model.id)
    elif response.status == "failed":
        print("Fine-tuning failed. Check the error details.")
        print("Error Details:", response.error)
    else:
        print("Fine-tuning is still in progress. Check again later.")

    return response.status

# Get the latest fine-tuning job for the specified model
latest_job = get_latest_fine_tune_job("davinci-SuperHero")  # Update with your model name

if latest_job:
    fine_tune_id = latest_job.id
    print("Latest Fine-tune ID:", fine_tune_id)

    # Check fine-tune status every 60 seconds until it is completed or fails
    while True:
        status = check_fine_tune_status(fine_tune_id)
        if status in ["succeeded", "failed"]:
            break
        time.sleep(60)  # Sleep for 60 seconds before checking again
else:
    print("No fine-tuning job found for the specified model.")


In [13]:
import openai
import time

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Function to retrieve the latest fine-tune job for a model
def get_latest_fine_tune_job(model_name):
    response = openai.FineTune.list(model=model_name, per_page=1)
    if response.data:
        return response.data[0]
    else:
        return None

# Function to check the status of a fine-tuning job
def check_fine_tune_status(fine_tune_id):
    response = openai.FineTune.retrieve(fine_tune_id)

    # Print relevant information about the fine-tuning job
    print("Fine-tune ID:", response.id)
    print("Status:", response.status)

    if response.status == "succeeded":
        print("Fine-tuning completed successfully!")
        print("Model ID:", response.model.id)
        print("Model Name:", response.model.name)  # Get the name of the model
    elif response.status == "failed":
        print("Fine-tuning failed. Check the error details.")
        print("Error Details:", response.error)
    else:
        print("Fine-tuning is still in progress. Check again later.")

    return response.status

# Get the latest fine-tuning job for the specified model
latest_job = get_latest_fine_tune_job("davinci-SuperHero")  # Update with your model name

if latest_job:
    fine_tune_id = latest_job.id
    print("Latest Fine-tune ID:", fine_tune_id)

    # Check fine-tune status every 60 seconds until it is completed or fails
    while True:
        status = check_fine_tune_status(fine_tune_id)
        if status in ["succeeded", "failed"]:
            break
        time.sleep(60)  # Sleep for 60 seconds before checking again
else:
    print("No fine-tuning job found for the specified model.")


Latest Fine-tune ID: ft-2eF6eh6kfyvDneXLWo1knwF5
Fine-tune ID: ft-2eF6eh6kfyvDneXLWo1knwF5
Status: succeeded
Fine-tuning completed successfully!


AttributeError: 'str' object has no attribute 'id'

In [20]:
import openai
import time

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Function to retrieve the latest fine-tune job for a model
def get_latest_fine_tune_job(model_name):
    response = openai.FineTune.list(model=model_name, per_page=1)
    if response.data:
        return response.data[0]
    else:
        return None

# Function to check the status of a fine-tuning job
def check_fine_tune_status(fine_tune_id):
    response = openai.FineTune.retrieve(fine_tune_id)

    # Print relevant information about the fine-tuning job
    print("Fine-tune ID:", response.id)
    print("Status:", response.status)

    if response.status == "succeeded":
        print("Fine-tuning completed successfully!")
        print("Model Name:", response.model)  # Get the name of the model
    elif response.status == "failed":
        print("Fine-tuning failed. Check the error details.")
        print("Error Details:", response.error)
    else:
        print("Fine-tuning is still in progress. Check again later.")

    return response.status

# Get the latest fine-tuning job for the specified model
latest_job = get_latest_fine_tune_job("davinci-SuperHero")  # Update with your model name

if latest_job:
    fine_tune_id = latest_job.id
    print("Latest Fine-tune ID:", fine_tune_id)

    # Check fine-tune status every 60 seconds until it is completed or fails
    while True:
        status = check_fine_tune_status(fine_tune_id)
        if status in ["succeeded", "failed"]:
            break
        time.sleep(60)  # Sleep for 60 seconds before checking again
else:
    print("No fine-tuning job found for the specified model.")


Latest Fine-tune ID: ft-2eF6eh6kfyvDneXLWo1knwF5
Fine-tune ID: ft-2eF6eh6kfyvDneXLWo1knwF5
Status: succeeded
Fine-tuning completed successfully!
Model Name: davinci


In [12]:
# Checking an older model 

import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(
  model="davinci:ft-hal149:superhero-2023-10-04-09-56-47",
  prompt="18, man, inmortality ->\n",
  temperature=0.7,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["END"]
)

text = response["choices"][0]["text"]
print(text)


This tall, broad-shouldered man has a strong jaw and piercing blue eyes. His dark hair is always perfectly groomed and a hint of stubble is always visible on his strong chin. He wears a black leather jacket over a white shirt and dark jeans. He has a confident air about him, like he can take on the world and win. He has the power of immortality, meaning he will never age or die. He has been around for centuries, watching the world change and evolve. He has seen empires rise and fall, witnessed science and technology become more advanced, and 


In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(
  model="davinci:ft-2eF6eh6kfyvDneXLWo1knwF5",
  prompt="18, man, inmortality ->\n",
  temperature=0.7,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["END"]
)

text = response["choices"][0]["text"]
print(text)

